In [ ]:
# import necessary packages
from gmodels.lwfchain import LWFChainGraph
from gmodels.gtypes.edge import Edge, EdgeType
from gmodels.factor import Factor
from gmodels.randomvariable import NumCatRVariable


In [ ]:
# define data and nodes
idata = {"outcome-values": [True, False]}
A = NumCatRVariable(
       node_id="A", input_data=idata, marginal_distribution=lambda x: 0.5
)
B = NumCatRVariable(
       node_id="B", input_data=idata, marginal_distribution=lambda x: 0.5
)
C = NumCatRVariable(
       node_id="C", input_data=idata, marginal_distribution=lambda x: 0.5
)
D = NumCatRVariable(
       node_id="D", input_data=idata, marginal_distribution=lambda x: 0.5
)
E = NumCatRVariable(
       node_id="E", input_data=idata, marginal_distribution=lambda x: 0.5
)
F = NumCatRVariable(
       node_id="F", input_data=idata, marginal_distribution=lambda x: 0.5
)
G = NumCatRVariable(
       node_id="G", input_data=idata, marginal_distribution=lambda x: 0.5
)
H = NumCatRVariable(
       node_id="H", input_data=idata, marginal_distribution=lambda x: 0.5
)
I = NumCatRVariable(
       node_id="I", input_data=idata, marginal_distribution=lambda x: 0.5
)
K = NumCatRVariable(
       node_id="K", input_data=idata, marginal_distribution=lambda x: 0.5
)
L = NumCatRVariable(
       node_id="L", input_data=idata, marginal_distribution=lambda x: 0.5
)


In [ ]:
# define edges
#
#  Cowell 2005, p. 110
#
#   A                      E---+
#   |                          |
#   +----+                 F <-+
#        |                 |
#   B <--+---> C --> D <---+
#   |                |
#   +---> H <--------+----> G
#   |     |
#   +---> I
#
AB_c = Edge(
  edge_id="AB",
  start_node=A,
  end_node=B,
  edge_type=EdgeType.DIRECTED,
)
AC_c = Edge(
  edge_id="AC",
  start_node=A,
  end_node=C,
  edge_type=EdgeType.DIRECTED,
)
CD_c = Edge(
  edge_id="CD",
  start_node=C,
  end_node=D,
  edge_type=EdgeType.DIRECTED,
)
EF_c = Edge(
  edge_id="EF",
  start_node=E,
  end_node=F,
  edge_type=EdgeType.DIRECTED,
)
FD_c = Edge(
  edge_id="FD",
  start_node=F,
  end_node=D,
  edge_type=EdgeType.DIRECTED,
)
DG_c = Edge(
  edge_id="DG",
  start_node=D,
  end_node=G,
  edge_type=EdgeType.DIRECTED,
)
DH_c = Edge(
  edge_id="DH",
  start_node=D,
  end_node=H,
  edge_type=EdgeType.DIRECTED,
)
BH_c = Edge(
  edge_id="BH",
  start_node=B,
  end_node=H,
  edge_type=EdgeType.DIRECTED,
)
BI_c = Edge(
  edge_id="BI",
  start_node=B,
  end_node=I,
  edge_type=EdgeType.DIRECTED,
)
HI_c = Edge(
  edge_id="HI",
  start_node=H,
  end_node=I,
  edge_type=EdgeType.UNDIRECTED,
)

In [ ]:
# define factor functions

def phi_e(scope_product):
    "Visit to Asia factor p(a)"
    ss = set(scope_product)
    if ss == set([("E", True)]):
        return 0.01
    elif ss == set([("E", False)]):
        return 0.99
    else:
        raise ValueError("Unknown scope product")

def phi_fe(scope_product):
    "Tuberculosis | Visit to Asia factor p(t,a)"
    ss = set(scope_product)
    if ss == set([("F", True), ("E", True)]):
        return 0.05
    elif ss == set([("F", False), ("E", True)]):
        return 0.95
    elif ss == set([("F", True), ("E", False)]):
        return 0.01
    elif ss == set([("F", False), ("E", False)]):
        return 0.99
    else:
        raise ValueError("Unknown scope product")


def phi_dg(scope_product):
    "either tuberculosis or lung cancer | x ray p(e,x)"
    ss = set(scope_product)
    if ss == set([("D", True), ("G", True)]):
        return 0.98
    elif ss == set([("D", False), ("G", True)]):
        return 0.05
    elif ss == set([("D", True), ("G", False)]):
        return 0.02
    elif ss == set([("D", False), ("G", False)]):
        return 0.95
    else:
        raise ValueError("Unknown scope product")

def phi_a(scope_product):
    "smoke factor p(s)"
    ss = set(scope_product)
    if ss == set([("A", True)]):
        return 0.5
    elif ss == set([("A", False)]):
        return 0.5
    else:
        raise ValueError("Unknown scope product")


def phi_ab(scope_product):
    "smoke given bronchitis p(s,b)"
    ss = set(scope_product)
    if ss == set([("A", True), ("B", True)]):
        return 0.6
    elif ss == set([("A", False), ("B", True)]):
        return 0.3
    elif ss == set([("A", True), ("B", False)]):
        return 0.4
    elif ss == set([("A", False), ("B", False)]):
        return 0.7
    else:
        raise ValueError("Unknown scope product")


def phi_ac(scope_product):
    "lung cancer given smoke p(s,l)"
    ss = set(scope_product)
    if ss == set([("A", True), ("C", True)]):
        return 0.1
    elif ss == set([("A", False), ("C", True)]):
        return 0.01
    elif ss == set([("A", True), ("C", False)]):
        return 0.9
    elif ss == set([("A", False), ("C", False)]):
        return 0.99
    else:
        raise ValueError("Unknown scope product")


def phi_cdf(scope_product):
    "either tuberculosis or lung given lung cancer and tuberculosis p(e, l, t)"
    ss = set(scope_product)
    if ss == set([("C", True), ("D", True), ("F", True)]):
        return 1
    elif ss == set([("C", True), ("D", False), ("F", True)]):
        return 0
    elif ss == set([("C", False), ("D", True), ("F", True)]):
        return 1
    elif ss == set([("C", False), ("D", False), ("F", True)]):
        return 0
    elif ss == set([("C", True), ("D", True), ("F", False)]):
        return 1
    elif ss == set([("C", True), ("D", False), ("F", False)]):
        return 0
    elif ss == set([("C", False), ("D", True), ("F", False)]):
        return 0
    elif ss == set([("C", False), ("D", False), ("F", False)]):
        return 1
    else:
        raise ValueError("Unknown scope product")


def phi_ihb(scope_product):
    "cough, dyspnoea, bronchitis I, H, B p(c,d,b)"
    ss = set(scope_product)
    if ss == set([("H", True), ("I", True), ("B", True)]):
        return 16
    elif ss == set([("H", True), ("I", False), ("B", True)]):
        return 1
    elif ss == set([("H", False), ("I", True), ("B", True)]):
        return 4
    elif ss == set([("H", False), ("I", False), ("B", True)]):
        return 1
    elif ss == set([("H", True), ("I", True), ("B", False)]):
        return 2
    elif ss == set([("H", True), ("I", False), ("B", False)]):
        return 1
    elif ss == set([("H", False), ("I", True), ("B", False)]):
        return 1
    elif ss == set([("H", False), ("I", False), ("B", False)]):
        return 1
    else:
        raise ValueError("Unknown scope product")


def phi_hbd(scope_product):
    "cough, either tuberculosis or lung cancer, bronchitis D, H, B p(c,b,e)"
    ss = set(scope_product)
    if ss == set([("H", True), ("D", True), ("B", True)]):
        return 5
    elif ss == set([("H", True), ("D", False), ("B", True)]):
        return 2
    elif ss == set([("H", False), ("D", True), ("B", True)]):
        return 1
    elif ss == set([("H", False), ("D", False), ("B", True)]):
        return 1
    elif ss == set([("H", True), ("D", True), ("B", False)]):
        return 3
    elif ss == set([("H", True), ("D", False), ("B", False)]):
        return 1
    elif ss == set([("H", False), ("D", True), ("B", False)]):
        return 1
    elif ss == set([("H", False), ("D", False), ("B", False)]):
        return 1
    else:
        raise ValueError("Unknown scope product")


def phi_bd(scope_product):
    "bronchitis, either tuberculosis or lung cancer B, D p(b,e)"
    ss = set(scope_product)
    if ss == set([("B", True), ("D", True)]):
        return 1 / 90
    elif ss == set([("B", False), ("D", True)]):
        return 1 / 11
    elif ss == set([("B", True), ("D", False)]):
        return 1 / 39
    elif ss == set([("B", False), ("D", False)]):
        return 1 / 5
    else:
        raise ValueError("Unknown scope product")



In [ ]:
# instantiate factors with factor functions and implied random variables in scope

E_cf = Factor(gid="E_cf", scope_vars=set([E]), factor_fn=phi_e)
EF_cf = Factor(
    gid="EF_cf", scope_vars=set([E, F]), factor_fn=phi_fe
)
DG_cf = Factor(
    gid="DG_cf", scope_vars=set([D, G]), factor_fn=phi_dg
)
A_cf = Factor(gid="A_cf", scope_vars=set([A]), factor_fn=phi_a)
AB_cf = Factor(
    gid="AB_cf", scope_vars=set([A, B]), factor_fn=phi_ab
)
AC_cf = Factor(
    gid="AC_cf", scope_vars=set([A, C]), factor_fn=phi_ac
)
CDF_cf = Factor(
    gid="CDF_cf", scope_vars=set([D, C, F]), factor_fn=phi_cdf
)

IHB_cf = Factor(
    gid="IHB_cf", scope_vars=set([H, I, B]), factor_fn=phi_ihb
)

HBD_cf = Factor(
    gid="HBD_cf", scope_vars=set([H, D, B]), factor_fn=phi_hbd
)
BD_cf = Factor(
    gid="BD_cf", scope_vars=set([D, B]), factor_fn=phi_bd
)

In [ ]:
# instantiate lwf chain graph and make a query
cowell = LWFChainGraph(
    gid="cowell",
    nodes=set([A, B, C, D, E, F, G, H, I]),
    edges=set([AB_c, AC_c, CD_c, EF_c, FD_c, DG_c, DH_c, BH_c, BI_c, HI_c]),
    factors=set([E_cf, EF_cf, DG_cf, A_cf, AB_cf, AC_cf, CDF_cf, IHB_cf, HBD_cf, 
        BD_cf])
)
evidences = set([("E", True), ("A", True), ("G", False)])

final_factor, a = cowell.cond_prod_by_variable_elimination(
    set([B]), evidences
)

round(final_factor.phi_normal(set([("B", True)])), 4)
# 0.60

